In [0]:
! pip install tensorflow-gpu==1.13.1
import csv
import cv2
from sklearn.model_selection import train_test_split
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.advanced_activations import ReLU
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback

In [0]:
! rm -rf ./*
! wget https://www.dropbox.com/s/nicav0b03kvfrz3/BehavioralCloning.tar.gz
! tar -xzvf BehavioralCloning.tar.gz

In [0]:
! ls

In [0]:
def read_csv_data(csv_path):
    ''' Reads CSV data from simulator
        Uses only center images
    '''
    with open(csv_path, newline='') as f:
        csv_data = list(csv.reader(f, skipinitialspace=True, delimiter=',', quoting=csv.QUOTE_NONE))
    
    image_paths, angles = [], []
    for row in csv_data:
        angle = float(row[1])
        image_paths.append(row[0])
        angles.append(angle)

    return image_paths, angles

In [0]:
def create_data_generator(paths, angles, is_val_data, batch_size):
  while True:
      X, Y = [], []
      if is_val_data:
          for i, path in enumerate(paths):
              img = cv2.imread(path, 0)
              X.append(img)
              Y.append(angles[i])

              if len(X) == batch_size:
                  X = np.array(X).reshape(-1, 160, 320, 1)
                  X = X / 255.0
                  yield X, Y
                  X, Y = ([],[])

      else:
          for i, path in enumerate(paths):
              img = cv2.imread(path, 0)
              X.append(img)
              Y.append(angles[i])

              if len(X) == batch_size:
                  X = np.array(X).reshape(-1, 160, 320, 1)
                  X = X / 255.0
                  yield X, Y
                  X, Y = ([],[])
              
              elif abs(angles[i]) > 0.10:
                  img = cv2.flip(img, 1)
                  angle = -angles[i]
                  X.append(img)
                  Y.append(angle)

                  if len(X) == batch_size:
                      X = np.array(X).reshape(-1, 160, 320, 1)
                      X = X / 255.0
                      yield X, Y
                      X, Y = ([],[])

In [0]:
def get_data_generators(batch_size):
    csv_path = 'driving_log.csv'
    image_paths, angles = read_csv_data(csv_path)

    # X, Y = get_image_data(image_paths, angles)

    # Y = np.array(Y)
    # X = np.array(X).reshape(-1, 160, 320, 1)
    # X = X / 255.0
    
    paths_train, paths_test, angles_train, angles_test = train_test_split(image_paths, angles, test_size=0.20, shuffle=True)

    train_gen = create_data_generator(paths_train, angles_train, is_val_data=False, batch_size=batch_size)
    val_gen = create_data_generator(paths_test, angles_test, is_val_data=True, batch_size=batch_size)

    return train_gen, val_gen, int(len(paths_train) / batch_size), int(len(paths_test) / batch_size)

In [0]:
def create_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(160, 320, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    
    model.add(Conv2D(128, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(0.10))
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dropout(0.10))
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dropout(0.10))
    model.add(Dense(10))
    model.add(Activation('relu'))

    model.add(Dense(1))

    model.compile(optimizer=Adam(lr=0.0001), loss='mse')

    return model

In [0]:
def train_model(model, train_gen, val_gen, steps_train, steps_val):

    filepath = "model-v4-{epoch:02d}.h5"
    checkpoint = ModelCheckpoint(filepath) 

    history = model.fit_generator(train_gen, validation_data=val_gen, epochs=50, 
                                  steps_per_epoch=steps_train, callbacks=[checkpoint], validation_steps=steps_val)

In [0]:
train_gen, val_gen, steps_train, steps_val = get_data_generators(128)

model = create_model()



In [0]:
train_model(model, train_gen, val_gen, steps_train, steps_val)

In [0]:
! ls

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
upload = drive.CreateFile({'Clone': 'model-v4-04.h5'})
upload.SetContentFile('model-v4-04.h5')
upload.Upload()